In [1]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [2]:
! kaggle datasets download arbethi/diabetic-retinopathy-level-detection/download?datasetVersionNumber=3

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 166, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [3]:
! unzip diabetic-retinopathy-level-detection.zip

unzip:  cannot find or open diabetic-retinopathy-level-detection.zip, diabetic-retinopathy-level-detection.zip.zip or diabetic-retinopathy-level-detection.zip.ZIP.


In [4]:
imageSize = [299,299]
trainPath = r"/content/preprocessed dataset/preprocessed dataset/training"
testPath = r"/content/preprocessed dataset/preprocessed dataset/testing"

In [5]:
from tensorflow.keras.layers import Dense, Flatten, Input 
from tensorflow.keras.models import Model 
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img 
from tensorflow.keras.applications.xception import Xception, preprocess_input 
from glob import glob 
import numpy as np 
import matplotlib.pyplot as plt

In [6]:
train_datagen = ImageDataGenerator (rescale = 1./255,
                                    shear_range = 0.2, 
                                    zoom_range = 0.2, 
                                    horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [7]:
training_set = train_datagen.flow_from_directory('/content/preprocessed dataset/preprocessed dataset/training',
                                                 target_size = (299, 299), 
                                                 batch_size = 32,
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('/content/preprocessed dataset/preprocessed dataset/testing',
                                            target_size = (299, 299), 
                                            batch_size = 32,
                                            class_mode = 'categorical')

FileNotFoundError: ignored

In [ ]:
xception = Xception(input_shape=imageSize + [3], weights='imagenet', include_top=False)
# don't train existing weights for layer in xception.layers:
for layer in xception.layers:
  layer.trainable = False
# our Layers (- you can add more if you want x = Flatten(/(xception.output)
x = Flatten()(xception.output)


In [ ]:
prediction = Dense(5, activation='softmax')(x)
model = Model(inputs=xception.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
r=model.fit_generator(training_set,validation_data=test_set,epochs=30,steps_per_epoch=len(training_set)//32,validation_steps=len(test_set)//32)

In [ ]:
model.save('Updated-Xception-diabetic-retinopathy.h5')